In [1]:
import pandas as pd
import numpy as np

## Institutes

In [2]:
df = pd.read_csv('../outputs/clean/institutes.csv')
df = df.drop_duplicates()
df.head()

,id istituto,nome istituto,tipo istituto,posti regolamentari,posti non disponibili,totale detenuti,polizia penitenziaria - effettivi,polizia penitenziaria - previsti,amministrativi - effettivi,amministrativi - previsti,...,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche,dati aggiornati al,personale polizia penitenziaria aggiornato al,personale amministrativo aggiornato al,data di aggiornamento spazi detentivi
0,MII181125,Siracusa,Casa circondariale,545.0,0.0,674.0,227.0,234.0,21.0,27.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
1,MII173764,Cuneo,Casa circondariale,433.0,54.0,400.0,191.0,206.0,14.0,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
2,MII181820,Vasto,Casa di lavoro,197.0,50.0,93.0,65.0,95.0,15.0,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
3,MII181839,Velletri,Casa circondariale,412.0,26.0,514.0,197.0,275.0,26.0,32.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
4,MII179932,Porto Azzurro,Casa di reclusione - Pasquale De Santis,334.0,13.0,311.0,160.0,205.0,23.0,26.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN


### Remove outliers

In [3]:
df.head()

,id istituto,nome istituto,tipo istituto,posti regolamentari,posti non disponibili,totale detenuti,polizia penitenziaria - effettivi,polizia penitenziaria - previsti,amministrativi - effettivi,amministrativi - previsti,...,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche,dati aggiornati al,personale polizia penitenziaria aggiornato al,personale amministrativo aggiornato al,data di aggiornamento spazi detentivi
0,MII181125,Siracusa,Casa circondariale,545.0,0.0,674.0,227.0,234.0,21.0,27.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
1,MII173764,Cuneo,Casa circondariale,433.0,54.0,400.0,191.0,206.0,14.0,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
2,MII181820,Vasto,Casa di lavoro,197.0,50.0,93.0,65.0,95.0,15.0,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
3,MII181839,Velletri,Casa circondariale,412.0,26.0,514.0,197.0,275.0,26.0,32.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
4,MII179932,Porto Azzurro,Casa di reclusione - Pasquale De Santis,334.0,13.0,311.0,160.0,205.0,23.0,26.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN


In [4]:
df['dati aggiornati al'] = pd.to_datetime(df['dati aggiornati al'])
df = df.sort_values(by='dati aggiornati al')

In [5]:
# 1. Group by 'dati aggiornati al' and sum 'posti regolamentari'
daily_total_posti = df.groupby('dati aggiornati al')['posti regolamentari'].sum()
daily_total_posti

dati aggiornati al
2024-10-05    51196.0
2024-10-06    51196.0
2024-10-07    51196.0
2024-10-09    51195.0
2024-10-10    51195.0
               ...   
2025-12-04    51276.0
2025-12-05    51276.0
2025-12-06    51276.0
2025-12-07    51276.0
2025-12-08    50258.0
Name: posti regolamentari, Length: 336, dtype: float64

In [6]:
# 2. Calculate the percentage difference from the previous day's total
# pct_change() is perfect for this.
daily_total_posti_diff_pct = daily_total_posti.pct_change().abs()
daily_total_posti_diff_pct.tail(20)

dati aggiornati al
2025-10-23    0.000000
2025-10-24    0.000000
2025-10-25    0.000000
2025-10-26    0.000000
2025-10-27    0.000000
2025-10-28    0.000000
2025-11-24    0.000488
2025-11-25    0.000000
2025-11-26    0.000020
2025-11-27    0.000020
2025-11-28    0.000000
2025-11-29    0.000020
2025-11-30    0.000000
2025-12-01    0.000000
2025-12-02    0.000000
2025-12-04    0.000020
2025-12-05    0.000000
2025-12-06    0.000000
2025-12-07    0.000000
2025-12-08    0.019853
Name: posti regolamentari, dtype: float64

In [7]:
# Define the percentage threshold for outlier
percentage_threshold = 0.30

In [8]:
bad_dates = daily_total_posti_diff_pct[daily_total_posti_diff_pct > percentage_threshold].index.tolist()
bad_dates

[]

In [9]:
# Removing false positives
# This assumes the pattern: [True Outlier, False Positive, True Outlier, False Positive, ...]
dates_to_remove = [date for i, date in enumerate(bad_dates) if i % 2 == 0]
dates_to_remove

[]

In [10]:
df_cleaned = df[~df['dati aggiornati al'].isin(dates_to_remove)].copy()
df_cleaned

,id istituto,nome istituto,tipo istituto,posti regolamentari,posti non disponibili,totale detenuti,polizia penitenziaria - effettivi,polizia penitenziaria - previsti,amministrativi - effettivi,amministrativi - previsti,...,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche,dati aggiornati al,personale polizia penitenziaria aggiornato al,personale amministrativo aggiornato al,data di aggiornamento spazi detentivi
0,MII181125,Siracusa,Casa circondariale,545.0,0.0,674.0,227.0,234.0,21.0,27.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
121,MII181377,Taranto,Casa circondariale - Carmelo Magli,500.0,17.0,952.0,309.0,297.0,25.0,31.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
122,MII172332,Biella,Casa circondariale,394.0,4.0,432.0,199.0,210.0,15.0,26.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
123,MII179306,Palermo Pagliarelli,Casa circondariale - Antonio Lorusso,1165.0,9.0,1378.0,661.0,581.0,45.0,51.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
124,MII179290,Padova,Casa circondariale,188.0,4.0,215.0,133.0,142.0,17.0,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63611,MII180348,Roma Rebibbia femminile,Casa circondariale - Germana Stefanini,272.0,28.0,369.0,221.0,214.0,20.0,26.0,...,94.0,87.0,8.0,132.0,171.0,169.0,2025-12-08,2025-10-31,2025-10-31,2025-12-08
63612,MII181870,Verbania,Casa circondariale,53.0,1.0,75.0,45.0,44.0,5.0,13.0,...,34.0,5.0,1.0,37.0,37.0,35.0,2025-12-08,2025-10-31,2025-10-31,2025-12-08
63613,MII179299,Padova Nuovo Complesso,Casa di reclusione,432.0,21.0,644.0,311.0,336.0,28.0,30.0,...,101.0,29.0,6.0,382.0,53.0,53.0,2025-12-08,2025-10-31,2025-10-31,2025-12-08
63604,MII179331,Paliano,Casa di reclusione,153.0,0.0,57.0,60.0,66.0,11.0,14.0,...,46.0,11.0,0.0,57.0,55.0,56.0,2025-12-08,2025-10-31,2025-10-31,2025-12-08


In [11]:
df_cleaned.to_csv('../outputs/clean/institutes.csv', index=False, encoding='UTF-8-sig')
df = pd.read_csv('../outputs/clean/institutes.csv')


## Most recent data

In [12]:
# Get the most recent update
df['dati aggiornati al'] = pd.to_datetime(df['dati aggiornati al'], format='%Y-%m-%d')
df_most_recent = df.loc[df.groupby('id istituto')['dati aggiornati al'].idxmax()]

# Adding columns for places available and overcrowding index
df_most_recent['posti disponibili'] = df_most_recent['posti regolamentari'] - df_most_recent['posti non disponibili']
df_most_recent['tasso di affollamento'] = round(((df_most_recent['totale detenuti'] / df_most_recent['posti disponibili'])*100),0)

In [13]:
df_most_recent.value_counts(['dati aggiornati al'])

dati aggiornati al
2025-12-08            185
2025-12-07              5
Name: count, dtype: int64

In [14]:
df_most_recent.columns

Index(['id istituto', 'nome istituto', 'tipo istituto', 'posti regolamentari',
       'posti non disponibili', 'totale detenuti',
       'polizia penitenziaria - effettivi', 'polizia penitenziaria - previsti',
       'amministrativi - effettivi', 'amministrativi - previsti',
       'educatori - effettivi', 'educatori - previsti', 'numero complessivo',
       'numero non disponibili', 'doccia', 'bidet', 'portatori di handicap',
       'servizi igienici con porta', 'accensione luce autonoma',
       'prese elettriche', 'dati aggiornati al',
       'personale polizia penitenziaria aggiornato al',
       'personale amministrativo aggiornato al',
       'data di aggiornamento spazi detentivi', 'posti disponibili',
       'tasso di affollamento'],
      dtype='object')

In [15]:
# Adding institutes' information for mapping the institutes
# Read institutes' csv

df_info = pd.read_csv('../outputs/clean/institutes_info.csv')
df_info = df_info.rename(columns={'id_istituto': 'id istituto'})

merged_df = pd.merge(df_most_recent, df_info, on='id istituto')

# Adding additional columns
merged_df['posti disponibili'] = merged_df['posti regolamentari'] - merged_df['posti non disponibili']

merged_df['tasso di affollamento'] = round((merged_df['totale detenuti'] / merged_df['posti disponibili']) * 100, 0)


# Keep only relevant columns
df_filtered = merged_df[
  [
    'id istituto',
    'nome istituto',
    'tasso di affollamento',
    'indirizzo',
    'tipo istituto',
   'posti regolamentari',
    'posti non disponibili',
    'posti disponibili',
    'totale detenuti',
    'dati aggiornati al',
    'polizia penitenziaria - previsti',
    'polizia penitenziaria - effettivi',
    'personale polizia penitenziaria aggiornato al',
    'amministrativi - effettivi',
    'amministrativi - previsti',
    'personale amministrativo aggiornato al',
    'numero complessivo',
    'numero non disponibili',
    'doccia',
    'bidet',
    'portatori di handicap',
    'servizi igienici con porta',
    'accensione luce autonoma',
    'prese elettriche',
    'data di aggiornamento spazi detentivi',
    'latitudine',
    'longitude'
    ]
    ]

df_filtered.head(2)

,id istituto,nome istituto,tasso di affollamento,indirizzo,tipo istituto,posti regolamentari,posti non disponibili,posti disponibili,totale detenuti,dati aggiornati al,...,numero non disponibili,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche,data di aggiornamento spazi detentivi,latitudine,longitude
0,MII152284,Genova Marassi,128.0,Piazzale Marassi n.2 - 16139 Genova,Casa circondariale,535.0,3.0,532.0,682.0,2025-12-08,...,2.0,128.0,175.0,3.0,213.0,100.0,0.0,2025-12-08,44.417913,8.951100
1,MII157783,Alba,109.0,Loc.Toppino - Via Vivaro n. 14 - 12051 Alba,Casa di reclusione - Giuseppe Montalto,138.0,91.0,47.0,51.0,2025-12-08,...,87.0,22.0,112.0,1.0,112.0,22.0,22.0,2025-12-08,44.691542,8.025636


In [16]:
df_filtered['scheda istituto'] = '<a href="https://www.giustizia.it/giustizia/page/it/dettaglio_scheda_istituto_penitenziario?s=' + df_filtered['id istituto'] + '">Vai alla scheda istituto</a>'


df_filtered.head()

/tmp/ipykernel_8602/3401019820.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['scheda istituto'] = '<a href="https://www.giustizia.it/giustizia/page/it/dettaglio_scheda_istituto_penitenziario?s=' + df_filtered['id istituto'] + '">Vai alla scheda istituto</a>'


,id istituto,nome istituto,tasso di affollamento,indirizzo,tipo istituto,posti regolamentari,posti non disponibili,posti disponibili,totale detenuti,dati aggiornati al,...,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche,data di aggiornamento spazi detentivi,latitudine,longitude,scheda istituto
0,MII152284,Genova Marassi,128.0,Piazzale Marassi n.2 - 16139 Genova,Casa circondariale,535.0,3.0,532.0,682.0,2025-12-08,...,128.0,175.0,3.0,213.0,100.0,0.0,2025-12-08,44.417913,8.951100,"<a href=""https://www.giustizia.it/giustizia/pa..."
1,MII157783,Alba,109.0,Loc.Toppino - Via Vivaro n. 14 - 12051 Alba,Casa di reclusione - Giuseppe Montalto,138.0,91.0,47.0,51.0,2025-12-08,...,22.0,112.0,1.0,112.0,22.0,22.0,2025-12-08,44.691542,8.025636,"<a href=""https://www.giustizia.it/giustizia/pa..."
2,MII158895,Milano San Vittore,229.0,Piazza Filangieri n.2 - 20123 Milano,Casa circondariale - Francesco Di Cataldo,702.0,218.0,484.0,1106.0,2025-12-08,...,272.0,184.0,0.0,327.0,325.0,325.0,2025-12-08,45.461598,9.166399,"<a href=""https://www.giustizia.it/giustizia/pa..."
3,MII158901,Belluno,130.0,Via Baldenich n.11 - 32100 Belluno,Casa circondariale,89.0,7.0,82.0,107.0,2025-12-08,...,20.0,20.0,0.0,50.0,40.0,40.0,2025-12-08,46.139738,12.219297,"<a href=""https://www.giustizia.it/giustizia/pa..."
4,MII158910,Agrigento,148.0,Piazza Di Lorenzo n. 1 - 92100 Agrigento,Casa circondariale - Pasquale Di Lorenzo,283.0,24.0,259.0,383.0,2025-12-08,...,192.0,42.0,0.0,247.0,228.0,0.0,2025-12-08,37.319422,13.617396,"<a href=""https://www.giustizia.it/giustizia/pa..."


In [17]:
# Create a copy first to avoid the warning
df_filtered = df_filtered.copy()

# Calculate metrics using loc for proper assignment
df_filtered.loc[:, 'stanze_disponibili'] = df_filtered['numero complessivo'] - df_filtered['numero non disponibili']

# Create list of columns to process
metrics = {
    'detenuti_stanza': lambda x: (x['totale detenuti'] / x['stanze_disponibili']).round(2),
    'polizia_pers': lambda x:  (x['polizia penitenziaria - effettivi']/ x['totale detenuti']).round(2),
    'doccia_pers': lambda x:  (x['totale detenuti'] / x['doccia']).round(2),
    'bidet_pers': lambda x:  (x['totale detenuti'] / x['bidet']).round(2),
    'servizi_pers': lambda x:  (x['totale detenuti'] / x['servizi igienici con porta']).round(2),
    'luci_pers': lambda x:  (x['totale detenuti'] / x['accensione luce autonoma']).round(2),
    'prese_pers': lambda x: (x['totale detenuti'] / x['prese elettriche']).round(2)
}

# Apply calculations
for col, func in metrics.items():
    df_filtered.loc[:, col] = func(df_filtered).replace([np.inf, -np.inf], np.nan)

In [18]:
df_filtered.sample(4)

,id istituto,nome istituto,tasso di affollamento,indirizzo,tipo istituto,posti regolamentari,posti non disponibili,posti disponibili,totale detenuti,dati aggiornati al,...,longitude,scheda istituto,stanze_disponibili,detenuti_stanza,polizia_pers,doccia_pers,bidet_pers,servizi_pers,luci_pers,prese_pers
15,MII159069,Asti,122.0,Frazione Quarto Inferiore n. 266 14100 Asti,Casa di reclusione,205.0,7.0,198.0,242.0,2025-12-08,...,8.206888,"<a href=""https://www.giustizia.it/giustizia/pa...",183.0,1.32,0.73,NaN,1.27,1.27,1.35,1.27
7,MII158935,Alghero,103.0,Via Vittorio Emanuele n.28 - 07041 Alghero,Casa di reclusione - Giuseppe Tomasiello,156.0,0.0,156.0,160.0,2025-12-08,...,8.315147,"<a href=""https://www.giustizia.it/giustizia/pa...",62.0,2.58,0.43,2.58,NaN,2.58,2.58,2.58
33,MII172827,Bergamo,184.0,via Monte Gleno n.61 - 24125 Bergamo,Casa circondariale,319.0,0.0,319.0,586.0,2025-12-08,...,9.754219,"<a href=""https://www.giustizia.it/giustizia/pa...",192.0,3.05,0.38,5.19,11.06,3.05,3.05,3.05
66,MII176451,Gela,167.0,C.da Balate SS 117 bis n. snc - 93012 Gela,Casa circondariale,48.0,0.0,48.0,80.0,2025-12-08,...,14.250245,"<a href=""https://www.giustizia.it/giustizia/pa...",48.0,1.67,0.62,1.67,1.67,1.67,80.00,1.74


In [19]:
# Saving csv
df_filtered.to_csv('../outputs/viz/institutes_most_recent.csv', index=False, encoding='UTF-8-sig')

## 1. Totals

In [20]:
grouped_df = df.groupby('dati aggiornati al').sum(numeric_only=True).reset_index()
grouped_df

,dati aggiornati al,posti regolamentari,posti non disponibili,totale detenuti,polizia penitenziaria - effettivi,polizia penitenziaria - previsti,amministrativi - effettivi,amministrativi - previsti,educatori - effettivi,educatori - previsti,numero complessivo,numero non disponibili,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche
0,2024-10-05,51196.0,4384.0,61880.0,31433.0,34035.0,3388.0,4049.0,969.0,1001.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2024-10-06,51196.0,4384.0,61912.0,31433.0,34035.0,3388.0,4049.0,969.0,1001.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2024-10-07,51196.0,4441.0,61846.0,31433.0,34035.0,3388.0,4049.0,969.0,1001.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2024-10-09,51195.0,4439.0,61863.0,31091.0,34035.0,3414.0,4079.0,978.0,1001.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2024-10-10,51195.0,4437.0,61843.0,31091.0,34035.0,3414.0,4079.0,978.0,1001.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,2025-12-04,51276.0,5127.0,63751.0,32386.0,34035.0,3317.0,4086.0,919.0,1003.0,31937.0,3299.0,14494.0,13768.0,444.0,30505.0,24633.0,21258.0
332,2025-12-05,51276.0,5077.0,63690.0,32386.0,34035.0,3317.0,4086.0,919.0,1003.0,31937.0,3282.0,14494.0,13768.0,444.0,30505.0,24633.0,21258.0
333,2025-12-06,51276.0,5080.0,63740.0,32386.0,34035.0,3317.0,4086.0,919.0,1003.0,31937.0,3285.0,14494.0,13768.0,444.0,30505.0,24633.0,21258.0
334,2025-12-07,51276.0,5080.0,63733.0,32386.0,34035.0,3317.0,4086.0,919.0,1003.0,31937.0,3285.0,14494.0,13768.0,444.0,30505.0,24633.0,21258.0


In [21]:
grouped_df['posti disponibili'] = (grouped_df['posti regolamentari'] - grouped_df['posti non disponibili']).round(0)
grouped_df['tasso di affollamento'] = (grouped_df['totale detenuti'] / grouped_df['posti disponibili'] * 100).round(4).astype(float)

grouped_df.head()


,dati aggiornati al,posti regolamentari,posti non disponibili,totale detenuti,polizia penitenziaria - effettivi,polizia penitenziaria - previsti,amministrativi - effettivi,amministrativi - previsti,educatori - effettivi,educatori - previsti,numero complessivo,numero non disponibili,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche,posti disponibili,tasso di affollamento
0,2024-10-05,51196.0,4384.0,61880.0,31433.0,34035.0,3388.0,4049.0,969.0,1001.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46812.0,132.1883
1,2024-10-06,51196.0,4384.0,61912.0,31433.0,34035.0,3388.0,4049.0,969.0,1001.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46812.0,132.2567
2,2024-10-07,51196.0,4441.0,61846.0,31433.0,34035.0,3388.0,4049.0,969.0,1001.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46755.0,132.2768
3,2024-10-09,51195.0,4439.0,61863.0,31091.0,34035.0,3414.0,4079.0,978.0,1001.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46756.0,132.3103
4,2024-10-10,51195.0,4437.0,61843.0,31091.0,34035.0,3414.0,4079.0,978.0,1001.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46758.0,132.2619


In [22]:
grouped_df = grouped_df[['dati aggiornati al', 'posti regolamentari', 'posti non disponibili', 'posti disponibili', 'totale detenuti', 'tasso di affollamento']]
# grouped_df['posti disponibili'] = grouped_df['posti regolamentari'] - grouped_df['posti non disponibili']
# grouped_df['tasso_affollamento'] = round((grouped_df['posti_occupati'] / grouped_df['posti_disponibili'])*100,4).astype(float)
grouped_df.head()


,dati aggiornati al,posti regolamentari,posti non disponibili,posti disponibili,totale detenuti,tasso di affollamento
0,2024-10-05,51196.0,4384.0,46812.0,61880.0,132.1883
1,2024-10-06,51196.0,4384.0,46812.0,61912.0,132.2567
2,2024-10-07,51196.0,4441.0,46755.0,61846.0,132.2768
3,2024-10-09,51195.0,4439.0,46756.0,61863.0,132.3103
4,2024-10-10,51195.0,4437.0,46758.0,61843.0,132.2619


In [23]:
grouped_df.tail(2)

,dati aggiornati al,posti regolamentari,posti non disponibili,posti disponibili,totale detenuti,tasso di affollamento
334,2025-12-07,51276.0,5080.0,46196.0,63733.0,137.9622
335,2025-12-08,50258.0,4887.0,45371.0,62555.0,137.8744


In [24]:
grouped_df.to_csv('../outputs/viz/institutes_totals.csv', index=False)

## Personale

In [25]:
df = pd.read_csv('../outputs/viz/institutes_most_recent.csv')
df.columns

Index(['id istituto', 'nome istituto', 'tasso di affollamento', 'indirizzo',
       'tipo istituto', 'posti regolamentari', 'posti non disponibili',
       'posti disponibili', 'totale detenuti', 'dati aggiornati al',
       'polizia penitenziaria - previsti', 'polizia penitenziaria - effettivi',
       'personale polizia penitenziaria aggiornato al',
       'amministrativi - effettivi', 'amministrativi - previsti',
       'personale amministrativo aggiornato al', 'numero complessivo',
       'numero non disponibili', 'doccia', 'bidet', 'portatori di handicap',
       'servizi igienici con porta', 'accensione luce autonoma',
       'prese elettriche', 'data di aggiornamento spazi detentivi',
       'latitudine', 'longitude', 'scheda istituto', 'stanze_disponibili',
       'detenuti_stanza', 'polizia_pers', 'doccia_pers', 'bidet_pers',
       'servizi_pers', 'luci_pers', 'prese_pers'],
      dtype='object')

In [26]:
df = pd.read_csv('../outputs/viz/institutes_most_recent.csv')


df_polizia = df[['nome istituto', 'totale detenuti', 'tasso di affollamento', 'polizia penitenziaria - previsti', 'polizia penitenziaria - effettivi', 'personale polizia penitenziaria aggiornato al', 'dati aggiornati al']]

df_polizia['polizia penitenziaria - mancante'] = df_polizia['polizia penitenziaria - previsti'] - df_polizia['polizia penitenziaria - effettivi']

df_polizia.head(2)

/tmp/ipykernel_8602/2362681573.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_polizia['polizia penitenziaria - mancante'] = df_polizia['polizia penitenziaria - previsti'] - df_polizia['polizia penitenziaria - effettivi']


,nome istituto,totale detenuti,tasso di affollamento,polizia penitenziaria - previsti,polizia penitenziaria - effettivi,personale polizia penitenziaria aggiornato al,dati aggiornati al,polizia penitenziaria - mancante
0,Genova Marassi,682.0,128.0,336.0,352.0,2025-10-31,2025-12-08,-16.0
1,Alba,51.0,109.0,107.0,84.0,2025-10-31,2025-12-08,23.0


In [27]:
df_polizia['polizia penitenziaria - mancante percentuale'] = round(df_polizia['polizia penitenziaria - mancante'] / df_polizia['polizia penitenziaria - previsti']*100,2)
df_polizia.head(2)


/tmp/ipykernel_8602/989766878.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_polizia['polizia penitenziaria - mancante percentuale'] = round(df_polizia['polizia penitenziaria - mancante'] / df_polizia['polizia penitenziaria - previsti']*100,2)


,nome istituto,totale detenuti,tasso di affollamento,polizia penitenziaria - previsti,polizia penitenziaria - effettivi,personale polizia penitenziaria aggiornato al,dati aggiornati al,polizia penitenziaria - mancante,polizia penitenziaria - mancante percentuale
0,Genova Marassi,682.0,128.0,336.0,352.0,2025-10-31,2025-12-08,-16.0,-4.76
1,Alba,51.0,109.0,107.0,84.0,2025-10-31,2025-12-08,23.0,21.50


In [28]:
df_polizia_clean = df_polizia[df_polizia['totale detenuti'] != 0]
df_polizia_clean.sort_values('polizia penitenziaria - mancante percentuale', ascending=False).head(2)

,nome istituto,totale detenuti,tasso di affollamento,polizia penitenziaria - previsti,polizia penitenziaria - effettivi,personale polizia penitenziaria aggiornato al,dati aggiornati al,polizia penitenziaria - mancante,polizia penitenziaria - mancante percentuale
138,Roma Rebibbia,303.0,113.0,189.0,132.0,2025-10-31,2025-12-08,57.0,30.16
175,Vasto,120.0,62.0,95.0,67.0,2025-10-31,2025-12-08,28.0,29.47


In [29]:
critical_prisons = df_polizia_clean[(df_polizia_clean['tasso di affollamento'] > 120) & (df_polizia_clean['polizia penitenziaria - mancante percentuale'] > 20)].reset_index(drop=True)
critical_prisons

,nome istituto,totale detenuti,tasso di affollamento,polizia penitenziaria - previsti,polizia penitenziaria - effettivi,personale polizia penitenziaria aggiornato al,dati aggiornati al,polizia penitenziaria - mancante,polizia penitenziaria - mancante percentuale
0,Brescia Verziano,123.0,173.0,95.0,75.0,2025-10-31,2025-12-08,20.0,21.05
1,Cassino,162.0,176.0,142.0,101.0,2025-10-31,2025-12-08,41.0,28.87
2,Matera,203.0,169.0,125.0,95.0,2025-10-31,2025-12-08,30.0,24.00
3,Napoli Poggioreale,2168.0,166.0,828.0,661.0,2025-10-31,2025-12-08,167.0,20.17
4,Porto Azzurro,387.0,121.0,205.0,151.0,2025-10-31,2025-12-08,54.0,26.34
5,Velletri,594.0,154.0,275.0,218.0,2025-10-31,2025-12-08,57.0,20.73


In [30]:
critical_prisons.to_csv('../outputs/viz/institutes_critical.csv', index=False, encoding='UTF-8-sig')

### Tasso Reale

In [31]:
df1 = pd.read_csv('../outputs/viz/bulletines_totals.csv')
df2 = pd.read_csv('../outputs/viz/institutes_totals.csv')

In [32]:
df2.tail(2)

,dati aggiornati al,posti regolamentari,posti non disponibili,posti disponibili,totale detenuti,tasso di affollamento
334,2025-12-07,51276.0,5080.0,46196.0,63733.0,137.9622
335,2025-12-08,50258.0,4887.0,45371.0,62555.0,137.8744


In [33]:
# Renaming columns for clarity and merging on a unified date column
df1.rename(columns={'Ultimo aggiornamento': 'Date', 'tasso_affollamento': 'tasso_affollamento_ufficiale'}, inplace=True)
df2.rename(columns={'dati aggiornati al': 'Date', 'tasso di affollamento': 'tasso_affollamento_reale'}, inplace=True)

In [34]:
# Converting the Date columns to datetime for consistency
df1['Date'] = pd.to_datetime(df1['Date'])
df2['Date'] = pd.to_datetime(df2['Date'])

In [35]:
df2.tail(2)

,Date,posti regolamentari,posti non disponibili,posti disponibili,totale detenuti,tasso_affollamento_reale
334,2025-12-07,51276.0,5080.0,46196.0,63733.0,137.9622
335,2025-12-08,50258.0,4887.0,45371.0,62555.0,137.8744


In [36]:
merged_df = pd.merge(df1[['Date', 'tasso_affollamento_ufficiale']], 
                     df2[['Date', 'tasso_affollamento_reale']], 
                     on='Date', 
                     how='outer')
merged_df.sort_values(by='Date', inplace=True)

merged_df.tail(2)

,Date,tasso_affollamento_ufficiale,tasso_affollamento_reale
404,2025-12-07,NaN,137.9622
405,2025-12-08,NaN,137.8744


In [37]:
filtered_df = merged_df[merged_df['Date'] > '2024-08-01']
filtered_df.tail(2)


,Date,tasso_affollamento_ufficiale,tasso_affollamento_reale
404,2025-12-07,NaN,137.9622
405,2025-12-08,NaN,137.8744


In [38]:
# Apply linear interpolation for missing values
filtered_df['tasso_affollamento_ufficiale (interpolated)'] = round((filtered_df['tasso_affollamento_ufficiale'].interpolate(method='linear')),4)
filtered_df['tasso_affollamento_reale (interpolated)'] = round((filtered_df['tasso_affollamento_reale'].interpolate(method='linear')),4)

filtered_df.tail(2)


/tmp/ipykernel_8602/3090938612.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['tasso_affollamento_ufficiale (interpolated)'] = round((filtered_df['tasso_affollamento_ufficiale'].interpolate(method='linear')),4)
/tmp/ipykernel_8602/3090938612.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['tasso_affollamento_reale (interpolated)'] = round((filtered_df['tasso_affollamento_reale'].interpolate(method='linear')),4)


,Date,tasso_affollamento_ufficiale,tasso_affollamento_reale,tasso_affollamento_ufficiale (interpolated),tasso_affollamento_reale (interpolated)
404,2025-12-07,NaN,137.9622,120.694,137.9622
405,2025-12-08,NaN,137.8744,120.694,137.8744


In [39]:
filtered_df.to_csv('../outputs/viz/tasso_affollamento.csv', index=False)

In [40]:
df_reale = filtered_df[['Date', 'tasso_affollamento_reale']]
df_reale = df_reale[df_reale['tasso_affollamento_reale'].notna()]
df_reale.head(2)

,Date,tasso_affollamento_reale
69,2024-10-05,132.1883
70,2024-10-06,132.2567


In [41]:
df_reale.to_csv('../outputs/viz/tasso_reale.csv', index=False)